<a href="https://colab.research.google.com/github/ibabedal/BLCA-analysis/blob/main/CNA_scaling_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# importing needed libraries

In [1]:
import os
import sys
import time
import psutil
import pandas as pd
import numpy  as np
import concurrent.futures
import datetime


from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# Downloading data from github

In [2]:
!wget https://raw.githubusercontent.com/ibabedal/BLCA-analysis/main/processing-mrna-cna.tar.gz
!tar zxf processing-mrna-cna.tar.gz

--2022-10-07 16:49:46--  https://raw.githubusercontent.com/ibabedal/BLCA-analysis/main/processing-mrna-cna.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21932951 (21M) [application/octet-stream]
Saving to: ‘processing-mrna-cna.tar.gz.2’

processing-mrna-cna 100%[===================>]  20.92M  --.-KB/s    in 0.1s    

2022-10-07 16:49:47 (157 MB/s) - ‘processing-mrna-cna.tar.gz.2’ saved [21932951/21932951]

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'


# Start CNA processing

In [3]:
df_cna = pd.read_csv('./CNA-TMB.csv')
df_cna.head()

,SAMPLE_ID,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A,ATAD3B,ATAD3C,AURKAIP1,B3GALT6,...,hsa-mir-361,hsa-mir-548m,hsa-mir-652,hsa-mir-220a,hsa-mir-513c,hsa-mir-513b,hsa-mir-513a-1,hsa-mir-513a-2,hsa-mir-224,TMB_CLASS
0,TCGA-2F-A9KO-01,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,2
1,TCGA-2F-A9KP-01,2,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,1
2,TCGA-2F-A9KQ-01,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
3,TCGA-2F-A9KR-01,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,0,0,0,0,-1,2
4,TCGA-2F-A9KT-01,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,2


In [4]:
x = df_cna.drop(['SAMPLE_ID', 'TMB_CLASS'], axis=1)
x.head()

,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A,ATAD3B,ATAD3C,AURKAIP1,B3GALT6,C1orf159,...,hsa-mir-1321,hsa-mir-361,hsa-mir-548m,hsa-mir-652,hsa-mir-220a,hsa-mir-513c,hsa-mir-513b,hsa-mir-513a-1,hsa-mir-513a-2,hsa-mir-224
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
1,2,2,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,0,0,0,0,-1
4,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [5]:
y = df_cna[['TMB_CLASS']]
y.head()

,TMB_CLASS
0,2
1,1
2,1
3,2
4,2


### Doing shifting and normalization

In [6]:


def do_something(x):
    '''
    do stuff
    '''
    pid  = os.getpid()
    ppid = os.getppid()
    start = time.time()
    print("PPID %s->%s Started"%(ppid,pid))
    #df['diff'] = datetime.datetime.now() - pd.to_datetime(df['Order Date'])
    for i in x.columns:
      x[i] = ((x[i] - x[i].min()) + 0.1)
    stop  = time.time()
    completed_in  = round(stop - start,2)
    return(x)


In [7]:
## parallel execution , credit to https://betterprogramming.pub/pandas-how-to-process-a-dataframe-in-parallel-make-pandas-lightning-fast-669978cf5356

logical    = False
df_results = []
num_procs  = psutil.cpu_count(logical=logical)
#if len(sys.argv) > 1:
#    num_procs = int(sys.argv[1])

big_dataframe = x.copy()
splitted_df = np.array_split(big_dataframe, num_procs)
start = time.time()
with concurrent.futures.ProcessPoolExecutor(max_workers=num_procs) as executor:
    results = [ executor.submit(do_something, x=df) for df in splitted_df ]
    for result in concurrent.futures.as_completed(results):
        try:
            df_results.append(result.result())
        except Exception as ex:
            print(str(ex))
            pass
end = time.time()
print("-------------------------------------------")
print("PPID %s Completed in %s"%(os.getpid(), round(end-start,2)))
df_results = pd.concat(df_results)

PPID 1063->1118 Started


KeyboardInterrupt: ignored

In [ ]:
x = df_results.copy()
x.head()

In [ ]:
## saving to file to avoid recacluation
x.to_csv('cna_features_shifted.csv',index=False)

In [8]:
x = pd.read_csv('cna_features_shifted.csv')
x.head()

,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A,ATAD3B,ATAD3C,AURKAIP1,B3GALT6,C1orf159,...,hsa-mir-1321,hsa-mir-361,hsa-mir-548m,hsa-mir-652,hsa-mir-220a,hsa-mir-513c,hsa-mir-513b,hsa-mir-513a-1,hsa-mir-513a-2,hsa-mir-224
0,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,...,2.1,2.1,1.1,2.1,1.1,2.1,2.1,2.1,2.1,1.1
1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,...,2.1,2.1,1.1,2.1,1.1,2.1,2.1,2.1,2.1,1.1
2,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,...,1.1,1.1,0.1,1.1,0.1,1.1,1.1,1.1,1.1,0.1
3,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,...,1.1,1.1,0.1,1.1,0.1,2.1,2.1,2.1,2.1,0.1
4,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,...,1.1,1.1,0.1,1.1,0.1,1.1,1.1,1.1,1.1,0.1


In [9]:
start = time.time()
for i in x.columns:
  sum_feature = x[i].sum()
  x[i] = x[i]/sum_feature
end = time.time()
print(round(end-start,2))
x.head()

8.05


,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A,ATAD3B,ATAD3C,AURKAIP1,B3GALT6,C1orf159,...,hsa-mir-1321,hsa-mir-361,hsa-mir-548m,hsa-mir-652,hsa-mir-220a,hsa-mir-513c,hsa-mir-513b,hsa-mir-513a-1,hsa-mir-513a-2,hsa-mir-224
0,0.001260,0.001260,0.001260,0.001260,0.001260,0.001260,0.001260,0.001260,0.001260,0.001260,...,0.002546,0.002546,0.002583,0.002498,0.002571,0.002477,0.002477,0.002477,0.002477,0.002473
1,0.004698,0.004698,0.004698,0.004698,0.004698,0.004698,0.004698,0.004698,0.004698,0.004698,...,0.002546,0.002546,0.002583,0.002498,0.002571,0.002477,0.002477,0.002477,0.002477,0.002473
2,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,...,0.001334,0.001334,0.000235,0.001308,0.000234,0.001297,0.001297,0.001297,0.001297,0.000225
3,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,...,0.001334,0.001334,0.000235,0.001308,0.000234,0.002477,0.002477,0.002477,0.002477,0.000225
4,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,...,0.001334,0.001334,0.000235,0.001308,0.000234,0.001297,0.001297,0.001297,0.001297,0.000225


### starting with Chi2

In [10]:
chi2_selector = SelectKBest(chi2, k=1000)
kbest_df = chi2_selector.fit_transform(x, y)
column_names = x.columns[chi2_selector.get_support()]

In [11]:
kbest_df_chi2=pd.DataFrame(kbest_df, columns = column_names, index=df_cna['SAMPLE_ID'])

kbest_df_chi2.head()

,GABPB2,SEMA6C,LYSMD1,SCNM1,TNFAIP8L2-SCNM1,TNFAIP8L2,TMOD4,VPS72,PIP5K1A,PSMD4,...,hsa-mir-3119-1,hsa-mir-378b,hsa-mir-3134,hsa-mir-3135,hsa-mir-3139,hsa-mir-548v,hsa-mir-548h-4,hsa-mir-302e,hsa-mir-3159,hsa-mir-220a
SAMPLE_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-2F-A9KO-01,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,...,0.003267,0.003513,0.003641,0.003792,0.000289,0.001758,0.001744,0.000391,0.000372,0.002571
TCGA-2F-A9KP-01,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,...,0.003267,0.005186,0.003641,0.003792,0.003181,0.001758,0.001744,0.004300,0.004092,0.002571
TCGA-2F-A9KQ-01,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,...,0.001711,0.001840,0.001907,0.001986,0.003181,0.003356,0.003329,0.004300,0.004092,0.000234
TCGA-2F-A9KR-01,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,...,0.003267,0.000167,0.000173,0.000181,0.003181,0.001758,0.001744,0.000391,0.000372,0.000234
TCGA-2F-A9KT-01,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,...,0.001711,0.001840,0.001907,0.001986,0.000289,0.001758,0.001744,0.000391,0.000372,0.000234


In [12]:
kbest_df_chi2.isnull().sum().sum()

0

In [13]:
kbest_df_chi2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 408 entries, TCGA-2F-A9KO-01 to TCGA-ZF-AA5P-01
Columns: 1000 entries, GABPB2 to hsa-mir-220a
dtypes: float64(1000)
memory usage: 3.1+ MB


In [14]:
kbest_df_chi2['TMB_CLASS'] = y.iloc[:,0].values

In [15]:
kbest_df_chi2.head()

,GABPB2,SEMA6C,LYSMD1,SCNM1,TNFAIP8L2-SCNM1,TNFAIP8L2,TMOD4,VPS72,PIP5K1A,PSMD4,...,hsa-mir-378b,hsa-mir-3134,hsa-mir-3135,hsa-mir-3139,hsa-mir-548v,hsa-mir-548h-4,hsa-mir-302e,hsa-mir-3159,hsa-mir-220a,TMB_CLASS
SAMPLE_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-2F-A9KO-01,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,...,0.003513,0.003641,0.003792,0.000289,0.001758,0.001744,0.000391,0.000372,0.002571,2
TCGA-2F-A9KP-01,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,...,0.005186,0.003641,0.003792,0.003181,0.001758,0.001744,0.004300,0.004092,0.002571,1
TCGA-2F-A9KQ-01,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,...,0.001840,0.001907,0.001986,0.003181,0.003356,0.003329,0.004300,0.004092,0.000234,1
TCGA-2F-A9KR-01,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,...,0.000167,0.000173,0.000181,0.003181,0.001758,0.001744,0.000391,0.000372,0.000234,2
TCGA-2F-A9KT-01,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,...,0.001840,0.001907,0.001986,0.000289,0.001758,0.001744,0.000391,0.000372,0.000234,2


In [16]:
kbest_df_chi2.to_csv('./CNA-TMB-Chi2.csv',index=False)

### Using information gain probability based method

In [17]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

In [18]:
ig_selector = SelectKBest(chi2, k=1000)
kbest_df = ig_selector.fit_transform(x, y)
column_names = x.columns[ig_selector.get_support()]

In [19]:
kbest_df_ig=pd.DataFrame(kbest_df, columns = column_names, index=df_cna['SAMPLE_ID'])

kbest_df_ig.head()

,GABPB2,SEMA6C,LYSMD1,SCNM1,TNFAIP8L2-SCNM1,TNFAIP8L2,TMOD4,VPS72,PIP5K1A,PSMD4,...,hsa-mir-3119-1,hsa-mir-378b,hsa-mir-3134,hsa-mir-3135,hsa-mir-3139,hsa-mir-548v,hsa-mir-548h-4,hsa-mir-302e,hsa-mir-3159,hsa-mir-220a
SAMPLE_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-2F-A9KO-01,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,...,0.003267,0.003513,0.003641,0.003792,0.000289,0.001758,0.001744,0.000391,0.000372,0.002571
TCGA-2F-A9KP-01,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,...,0.003267,0.005186,0.003641,0.003792,0.003181,0.001758,0.001744,0.004300,0.004092,0.002571
TCGA-2F-A9KQ-01,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,...,0.001711,0.001840,0.001907,0.001986,0.003181,0.003356,0.003329,0.004300,0.004092,0.000234
TCGA-2F-A9KR-01,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,...,0.003267,0.000167,0.000173,0.000181,0.003181,0.001758,0.001744,0.000391,0.000372,0.000234
TCGA-2F-A9KT-01,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,...,0.001711,0.001840,0.001907,0.001986,0.000289,0.001758,0.001744,0.000391,0.000372,0.000234


In [20]:
kbest_df_ig.isnull().sum().sum()

0

In [21]:
kbest_df_ig.info()

<class 'pandas.core.frame.DataFrame'>
Index: 408 entries, TCGA-2F-A9KO-01 to TCGA-ZF-AA5P-01
Columns: 1000 entries, GABPB2 to hsa-mir-220a
dtypes: float64(1000)
memory usage: 3.1+ MB


In [22]:
kbest_df_ig['TMB_CLASS'] = y.iloc[:,0].values

In [23]:
kbest_df_ig.head()

,GABPB2,SEMA6C,LYSMD1,SCNM1,TNFAIP8L2-SCNM1,TNFAIP8L2,TMOD4,VPS72,PIP5K1A,PSMD4,...,hsa-mir-378b,hsa-mir-3134,hsa-mir-3135,hsa-mir-3139,hsa-mir-548v,hsa-mir-548h-4,hsa-mir-302e,hsa-mir-3159,hsa-mir-220a,TMB_CLASS
SAMPLE_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-2F-A9KO-01,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,...,0.003513,0.003641,0.003792,0.000289,0.001758,0.001744,0.000391,0.000372,0.002571,2
TCGA-2F-A9KP-01,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,...,0.005186,0.003641,0.003792,0.003181,0.001758,0.001744,0.004300,0.004092,0.002571,1
TCGA-2F-A9KQ-01,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,...,0.001840,0.001907,0.001986,0.003181,0.003356,0.003329,0.004300,0.004092,0.000234,1
TCGA-2F-A9KR-01,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,0.001652,...,0.000167,0.000173,0.000181,0.003181,0.001758,0.001744,0.000391,0.000372,0.000234,2
TCGA-2F-A9KT-01,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,0.003154,...,0.001840,0.001907,0.001986,0.000289,0.001758,0.001744,0.000391,0.000372,0.000234,2


In [24]:
kbest_df_ig.to_csv('./CNA-TMB-IG.csv',index=False)

## using RFE (Recursive Feature Elimination)

In [35]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

In [37]:
rfe_selector = RFE(estimator=RandomForestClassifier(), n_features_to_select=1000, step=100, verbose=5)
rfe_selector.fit(x, y)
rfe_support = rfe_selector.get_support()

Fitting estimator with 25128 features.


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_rfe.py:283: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y, **fit_params)


Fitting estimator with 25028 features.


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_rfe.py:283: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y, **fit_params)


Fitting estimator with 24928 features.


...

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_rfe.py:283: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y, **fit_params)


Fitting estimator with 1128 features.


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_rfe.py:283: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y, **fit_params)


Fitting estimator with 1028 features.


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_rfe.py:283: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X[:, features], y, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_rfe.py:310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X[:, features], y, **fit_params)


In [41]:
rfe_feature = x.loc[:,rfe_support].columns.tolist()
print(rfe_feature)
print(len(rfe_feature))

['MIR34A', 'LINC02606', 'UBE4B', 'PGD', 'CORT', 'HNRNPCL1', 'HNRNPCL3', 'PRAMEF10', 'PRAMEF13', 'PRAMEF2', 'PRAMEF3', 'PRAMEF4', 'PRDM2', 'UQCRHL', 'SPEN', 'FAM131C', 'PADI4', 'DDOST', 'USP48', 'LACTBL1', 'ARID1A', 'COL16A1', 'TMEM234', 'KIAA0319L', 'AGO1', 'THRAP3', 'LSM10', 'CSF3R', 'LINC01137', 'CDCA8', 'MANEAL', 'MTF1', 'SF3A3', 'UTP11L', 'RHBDL2', 'SMAP2', 'C1orf185', 'NFIA', 'NEGR1', 'NEGR1-IT1', 'ST6GALNAC3', 'SAMD13', 'RPF1', 'CLCA4', 'CCDC18', 'TRMT13', 'LRRC39', 'CDC14A', 'AMY2A', 'C1orf194', 'AP4B1', 'ANKRD34A', 'HIST2H2BF', 'LIX1L', 'NBPF14', 'APH1A', 'PSMB4', 'PRR9', 'SCAMP3', 'ASH1L', 'IQGAP3', 'NES', 'HDGF', 'SH2D2A', 'FCRL3', 'CD1A', 'DARC', 'CRP', 'DUSP23', 'C1orf204', 'SLAMF9', 'PIGM', 'KCNJ10', 'NHLH1', 'LY9', 'ITLN2', 'USF1', 'PFDN2', 'NIT1', 'B4GALT3', 'PPOX', 'ADAMTS4', 'MIR5187', 'NR1I3', 'SDHC', 'C1orf192', 'FCGR2A', 'FCGR2B', 'FCGR2C', 'FCGR3B', 'HSPA7', 'NOS1AP', 'UHMK1', 'HSD17B7', 'RGS5', 'LOC440700', 'FMO9P', 'TADA1', 'GPA33', 'GM140', 'RGS16', 'C1orf21', '

In [63]:
kbest_df_rfe=x[rfe_feature].copy()
kbest_df_rfe['TMB_CLASS'] = y.iloc[:,0].values
kbest_df_rfe['SAMPLE_ID'] = df_cna['SAMPLE_ID'].copy()
kbest_df_rfe.set_index('SAMPLE_ID', inplace=True)
kbest_df_rfe.head()
#x[rfe_feature]

,MIR34A,LINC02606,UBE4B,PGD,CORT,HNRNPCL1,HNRNPCL3,PRAMEF10,PRAMEF13,PRAMEF2,...,hsa-mir-3181,hsa-mir-548h-3,hsa-mir-633,hsa-mir-548d-2,hsa-mir-924,hsa-mir-1302-11,hsa-mir-220b,hsa-mir-1289-1,hsa-mir-1302-5,TMB_CLASS
SAMPLE_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-2F-A9KO-01,0.000215,0.000215,0.001263,0.001266,0.001266,0.000217,0.000217,0.000217,0.000217,0.000217,...,0.002558,0.001602,0.001725,0.002025,0.001442,0.002735,0.002943,0.002803,0.003036,2
TCGA-2F-A9KP-01,0.006670,0.006670,0.004708,0.004719,0.004719,0.006727,0.006727,0.006727,0.006727,0.006727,...,0.002558,0.004514,0.001725,0.002025,0.002753,0.005340,0.002943,0.002803,0.003036,1
TCGA-2F-A9KQ-01,0.002367,0.002367,0.002412,0.002417,0.002417,0.002387,0.002387,0.002387,0.002387,0.002387,...,0.002558,0.003058,0.001725,0.002025,0.001442,0.002735,0.002943,0.001899,0.001590,1
TCGA-2F-A9KR-01,0.002367,0.002367,0.002412,0.002417,0.002417,0.002387,0.002387,0.002387,0.002387,0.002387,...,0.002558,0.001602,0.004860,0.003954,0.001442,0.002735,0.002943,0.002803,0.003036,2
TCGA-2F-A9KT-01,0.002367,0.002367,0.002412,0.002417,0.002417,0.002387,0.002387,0.002387,0.002387,0.002387,...,0.002558,0.001602,0.001725,0.002025,0.002753,0.002735,0.002943,0.000995,0.004481,2


In [64]:
kbest_df_ig.to_csv('./CNA-TMB-RFE.csv',index=False)

## comparing features

In [29]:
kbest_df_chi2.drop('TMB_CLASS', axis=1 ,inplace=True)
list_of_fea_chi2 = kbest_df_chi2.columns.to_list()
len(list_of_fea_chi2)

1000

In [30]:
kbest_df_ig.drop('TMB_CLASS', axis=1, inplace=True)
list_of_fea_ig = kbest_df_ig.columns.to_list()
len(list_of_fea_ig)

1000

In [47]:
list_of_fea_rfe = rfe_feature
print(len(list_of_fea_rfe))

1000


In [68]:
# compare Chi2 to IG
common_features_chi2_ig = set(list_of_fea_chi2).intersection(list_of_fea_ig)
print(list(common_features_chi2_ig))
print(len(list(common_features_chi2_ig)))

['MZB1', 'GORAB', 'SMPD1', 'OXNAD1', 'KCNJ10', 'OR52A1', 'SNRPD1', 'LYSMD1', 'LINC00852', 'MIR4270', 'ATG7', 'HBE1', 'POGZ', 'OR52N1', 'BEX5', 'LINC02153', 'C4orf51', 'AKIP1', 'DND1', 'NR2C2', 'PCDHA13', 'SLC22A18AS', 'PCDHA1', 'BIN3', 'PHLDA2', 'MBOAT4', 'UCP1', 'MROH9', 'CHCHD4', 'SMAD1', 'OR52N5', 'SELL', 'MIR320A', 'MIR1277', 'LGI3', 'NOS1AP', 'MIR133A1HG', 'PCDHGA4', 'FAM90A25P', 'SLAMF7', 'DNAJC18', 'MPZL1', 'TMOD4', 'OR56B1', 'FBLN2', 'NCR3LG1', 'MIR5197', 'OR51G2', 'RPL31P11', 'PCDHB4', 'CSRP3', 'NUF2', 'RFTN1', 'USP47', 'TUB', 'EMC3-AS1', 'IGIP', 'TRIM68', 'KRTAP5-4', 'TMEM9B', 'PCDHA9', 'ATP1A4', 'SCOC-AS1', 'hsa-mir-4259', 'HBD', 'THUMPD3', 'ELMOD2', 'DPYSL2', 'OR52E4', 'BLZF1', 'PCDH19', 'TNRC18P1', 'IQSEC1', 'LOC100507156', 'RBPMS', 'ATP1B1', 'UHMK1', 'OR51D1', 'MIR3926-1', 'TNFRSF10B', 'PCDHGC3', 'KCNQ1DN', 'HARS2', 'TSSC2', 'C11orf16', 'RHOXF1', 'ZDHHC2', 'PCM1', 'GAB1', 'NRG2', 'MIR597', 'VPS37A', 'OR52L1', 'GALNT15', 'COL6A4P1', 'ITLN1', 'ANKRD36BP1', 'FGF1', 'JAKMIP2-

In [67]:
# compare Chi2 to RFE
common_features_chi2_rfe = set(list_of_fea_chi2).intersection(list_of_fea_rfe)
print(list(common_features_chi2_rfe))
print(len(list(common_features_chi2_rfe)))

['hsa-mir-548v', 'USF1', 'KCNJ10', 'hsa-mir-3139', 'B4GALT3', 'TADA1', 'ATG7', 'DMRTA1', 'MIR4790', 'BRK1', 'FAM90A2P', 'HDAC11', 'ITPR1-AS1', 'RPUSD3', 'ADAMDEC1', 'hsa-mir-3134', 'PFDN2', 'NOS1AP', 'MIR133A1HG', 'WDR44', 'FAM183CP', 'SNORA7A', 'CIDEC', 'PPOX', 'SDHC', 'FCGR2A', 'DUSP23', 'CT47B1', 'HSPA7', 'FGD5-AS1', 'TAMM41', 'IQSEC1', 'C1orf204', 'UHMK1', 'hsa-mir-378b', 'KBTBD11', 'CDKN2B', 'NKX3-1', 'LRCH2', 'COL6A4P1', 'SRGAP3-AS3', 'MIR4791', 'RPL32', 'NR1I3', 'VSIG1', 'FMO9P', 'TMEM43', 'LINC01267', 'RNF122', 'LINC01239', 'HSD17B7', 'hsa-mir-3159', 'PSMD10', 'CUL4B', 'C1orf192', 'C1GALT1C1', 'RBMXL3', 'FGD5', 'ZBTB33', 'TMEM40', 'LOC401052', 'MSR1', 'SGCZ', 'TTLL3', 'BHLHE40', 'CT47A2', 'FCGR2B', 'TSEN2', 'ADAMTS4', 'CRP', 'AGTR2', 'hsa-mir-3135', 'FCGR3B', 'PIGM', 'CAND2', 'GPA33', 'KIF13B', 'ATP2B2', 'hsa-mir-302e', 'LOC440700', 'DPH3', 'NHLH1', 'RHOXF2B', 'NUP210', 'LSM3', 'ITLN2', 'SLAMF9', 'MTMR14', 'MIR3926-2', 'FCGR2C', 'OXTR', 'MIR5187', 'RGS5', 'LY9', 'hsa-mir-548h-4

In [50]:
# compare IG to RFE
common_features_ig_rfe = set(list_of_fea_ig).intersection(list_of_fea_rfe)
print(list(common_features_ig_rfe))
print(len(list(common_features_ig_rfe)))

['hsa-mir-548v', 'USF1', 'KCNJ10', 'hsa-mir-3139', 'B4GALT3', 'TADA1', 'ATG7', 'DMRTA1', 'MIR4790', 'BRK1', 'FAM90A2P', 'HDAC11', 'ITPR1-AS1', 'RPUSD3', 'ADAMDEC1', 'hsa-mir-3134', 'PFDN2', 'NOS1AP', 'MIR133A1HG', 'WDR44', 'FAM183CP', 'SNORA7A', 'CIDEC', 'PPOX', 'SDHC', 'FCGR2A', 'DUSP23', 'CT47B1', 'HSPA7', 'FGD5-AS1', 'TAMM41', 'IQSEC1', 'C1orf204', 'UHMK1', 'hsa-mir-378b', 'KBTBD11', 'CDKN2B', 'NKX3-1', 'LRCH2', 'COL6A4P1', 'SRGAP3-AS3', 'MIR4791', 'RPL32', 'NR1I3', 'VSIG1', 'FMO9P', 'TMEM43', 'LINC01267', 'RNF122', 'LINC01239', 'HSD17B7', 'hsa-mir-3159', 'PSMD10', 'CUL4B', 'C1orf192', 'C1GALT1C1', 'RBMXL3', 'FGD5', 'ZBTB33', 'TMEM40', 'LOC401052', 'MSR1', 'SGCZ', 'TTLL3', 'BHLHE40', 'CT47A2', 'FCGR2B', 'TSEN2', 'ADAMTS4', 'CRP', 'AGTR2', 'hsa-mir-3135', 'FCGR3B', 'PIGM', 'CAND2', 'GPA33', 'KIF13B', 'ATP2B2', 'hsa-mir-302e', 'LOC440700', 'DPH3', 'NHLH1', 'RHOXF2B', 'NUP210', 'LSM3', 'ITLN2', 'SLAMF9', 'MTMR14', 'MIR3926-2', 'FCGR2C', 'OXTR', 'MIR5187', 'RGS5', 'LY9', 'hsa-mir-548h-4

In [69]:
# common between all three methods
common_features = set(common_features_chi2_rfe).intersection(common_features_ig_rfe)
print(list(common_features))
print(len(list(common_features)))

['hsa-mir-548v', 'USF1', 'KCNJ10', 'hsa-mir-3139', 'TADA1', 'B4GALT3', 'DMRTA1', 'ATG7', 'MIR4790', 'BRK1', 'FAM90A2P', 'HDAC11', 'ITPR1-AS1', 'RPUSD3', 'ADAMDEC1', 'hsa-mir-3134', 'PFDN2', 'NOS1AP', 'MIR133A1HG', 'WDR44', 'FAM183CP', 'SNORA7A', 'CIDEC', 'PPOX', 'SDHC', 'FCGR2A', 'DUSP23', 'CT47B1', 'HSPA7', 'FGD5-AS1', 'TAMM41', 'IQSEC1', 'C1orf204', 'UHMK1', 'hsa-mir-378b', 'KBTBD11', 'CDKN2B', 'NKX3-1', 'LRCH2', 'COL6A4P1', 'SRGAP3-AS3', 'MIR4791', 'RPL32', 'NR1I3', 'VSIG1', 'FMO9P', 'TMEM43', 'LINC01267', 'RNF122', 'LINC01239', 'HSD17B7', 'hsa-mir-3159', 'PSMD10', 'CUL4B', 'C1orf192', 'C1GALT1C1', 'RBMXL3', 'FGD5', 'ZBTB33', 'TMEM40', 'LOC401052', 'MSR1', 'SGCZ', 'TTLL3', 'BHLHE40', 'CT47A2', 'FCGR2B', 'TSEN2', 'ADAMTS4', 'CRP', 'AGTR2', 'hsa-mir-3135', 'FCGR3B', 'PIGM', 'CAND2', 'GPA33', 'KIF13B', 'ATP2B2', 'hsa-mir-302e', 'LOC440700', 'DPH3', 'NHLH1', 'RHOXF2B', 'NUP210', 'LSM3', 'ITLN2', 'SLAMF9', 'MTMR14', 'MIR3926-2', 'FCGR2C', 'OXTR', 'MIR5187', 'RGS5', 'LY9', 'hsa-mir-548h-4

In [71]:
## Creating the CSV file from those features
df_final = x[common_features].copy()
df_final['TMB_CLASS'] = y.iloc[:,0].values
df_final['SAMPLE_ID'] = df_cna['SAMPLE_ID']
df_final.set_index('SAMPLE_ID', inplace=True)
df_final.head()

,hsa-mir-548v,USF1,KCNJ10,hsa-mir-3139,TADA1,B4GALT3,DMRTA1,ATG7,MIR4790,BRK1,...,LY9,hsa-mir-548h-4,TIMP4,RHOXF2,TEX13B,NIT1,SLC6A1-AS1,MKRN2,HRH1,TMB_CLASS
SAMPLE_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-2F-A9KO-01,0.001758,0.002934,0.003067,0.000289,0.003159,0.002942,0.000160,0.003484,0.003705,0.003490,...,0.002975,0.001744,0.003308,0.002571,0.002542,0.002926,0.003501,0.003319,0.003501,2
TCGA-2F-A9KP-01,0.001758,0.004331,0.003067,0.003181,0.003159,0.004343,0.001758,0.005143,0.005469,0.005151,...,0.004392,0.001744,0.004883,0.002571,0.002542,0.004319,0.005168,0.004899,0.005168,1
TCGA-2F-A9KQ-01,0.003356,0.001537,0.001606,0.003181,0.001655,0.001541,0.000160,0.001825,0.001941,0.001828,...,0.001559,0.003329,0.001733,0.000234,0.000231,0.001532,0.001834,0.001738,0.001834,1
TCGA-2F-A9KR-01,0.001758,0.001537,0.001606,0.003181,0.003159,0.001541,0.000160,0.000166,0.000176,0.000166,...,0.001559,0.001744,0.000158,0.000234,0.000231,0.001532,0.000167,0.000158,0.000167,2
TCGA-2F-A9KT-01,0.001758,0.004331,0.001606,0.000289,0.001655,0.004343,0.001758,0.001825,0.001941,0.001828,...,0.001559,0.001744,0.004883,0.000234,0.000231,0.004319,0.001834,0.001738,0.001834,2


In [72]:
df_final.to_csv('./CNA-TMB-Reduced.csv', index=False)

## using csv2arff to convert .csv to arff : https://pypi.org/project/csv2arff/

In [2]:
#!pip install numpy
#!pip install python-javabridge
#!pip install python-weka-wrapper3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement wekaexamples (from versions: none)
ERROR: No matching distribution found for wekaexamples


### installing the java runtime on the colab

In [3]:
!sudo apt-get update
!sudo apt-get install -y default-jdk 
!git clone https://github.com/heshida01/mrmd.git

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

In [4]:
# to get terminal on colab ##
#!pip install colab-xterm
#%load_ext colabxterm
#%xterm

In [5]:
# start JVM
#import weka.core.jvm as jvm
#jvm.start()
#jvm.stop()

DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['/usr/local/lib/python3.7/dist-packages/javabridge/jars/rhino-1.7R4.jar', '/usr/local/lib/python3.7/dist-packages/javabridge/jars/runnablequeue.jar', '/usr/local/lib/python3.7/dist-packages/javabridge/jars/cpython.jar', '/usr/local/lib/python3.7/dist-packages/weka/lib/weka.jar', '/usr/local/lib/python3.7/dist-packages/weka/lib/python-weka-wrapper.jar']
DEBUG:weka.core.jvm:MaxHeapSize=default
DEBUG:weka.core.jvm:Package support disabled


In [8]:
!pip install csv2arff

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for csv2arff: filename=csv2arff-2.0-py3-none-any.whl size=17252 sha256=737bd4116effbffed32c7a215a72c6f18d1540f073bf6cb55b7568eb09f5834b
  Stored in directory: /root/.cache/pip/wheels/e1/d9/93/908a6541f84dbfbb6df585613b96c8406e5e84896ef7968db1
Successfully built csv2arff


In [37]:
!csv2arff CNA-TMB-Chi2.csv CNA-TMB-Chi2.arff

In [38]:
# replacing numeric with list for class TMB so that mrmd can work on it
!grep TMB CNA-TMB-Chi2.arff
!sed -i 's/TMB_CLASS numeric/TMB_CLASS \{1,2\}/' CNA-TMB-Chi2.arff
!grep TMB CNA-TMB-Chi2.arff

@relation CNA-TMB-Chi2
@attribute TMB_CLASS numeric
@relation CNA-TMB-Chi2
@attribute TMB_CLASS {1,2}


In [39]:
!java -jar ./mrmd/mrmd.jar -i ./CNA-TMB-Chi2.arff -o ./CNA-TMB-MRMD.csv

Initialization...
Initialization over!!!
Start to calculate Pearson's correlation coefficient...
Calculating Pearson's correlation coefficient over!!!
Start to calculate Euclidean Distance...
Calculating Euclidean Distance over!!!
MRMD over.
Feature selction optimation begin
model:rf
feature num: 1 rate: 0.6274509803921569
feature num: 2 rate: 0.6274509803921569
feature num: 3 rate: 0.6274509803921569
feature num: 4 rate: 0.6274509803921569
feature num: 5 rate: 0.6274509803921569
feature num: 6 rate: 0.6274509803921569
feature num: 7 rate: 0.6323529411764706
feature num: 8 rate: 0.6348039215686274
feature num: 9 rate: 0.6372549019607843
feature num: 10 rate: 0.6519607843137255
feature num: 11 rate: 0.6519607843137255
feature num: 12 rate: 0.6495098039215687
feature num: 13 rate: 0.6519607843137255
feature num: 14 rate: 0.6507352941176471
feature num: 15 rate: 0.6507352941176471
feature num: 16 rate: 0.6519607843137255
feature num: 17 rate: 0.6507352941176471
feature num: 18 rate: 0.649

In [40]:
!head ./CNA-TMB-MRMD.csv

The number of selected features is: 1000

The index of selected features start from 0

NO.		FeaName		Score

1		Fea43		1.0
2		Fea42		1.0
3		Fea39		0.9962424241832826
4		Fea41		0.9887092184752435
